# Example 1: 2-D fault network extraction from a numerical model
This example describes how to extract a 2-D fault network from a numerical model simulating continental rifting. This example is based on the study of Naliboff et al. (2020).

## Setup

1. First, you will need sign in to your Google account. If you're not signed in, you can sign in [here](https://myaccount.google.com/?utm_source=sign_in_no_continue)
2. Next, head on to the [Colab Welcome Page](https://colab.research.google.com/notebooks/welcome.ipynb#recent=true).
3. There, select Github in the top tab and search for: https://github.com/thilowrona/fault_analysis_toolbox/blob/master/examples/1-fault_extraction/1-fault_extraction.ipynb
4. Clicking opens this notebook. This is a Jupyter notebook; an awesome combination of code and documentation allowing us work on, describe and share our projects.
5. When you run the first cell, you will face a pop-up saying ‘Warning: This notebook was not authored by Google’; you should click on ‘Run Anyway’ to get rid of the warning.
6. Next we want to save our notebook. If you click on ‘File’ and then ‘Save’, you will see a pop-up saying ´CANNOT SAVE CHANGES´. Now, click on ‘SAVE A COPY IN DRIVE’. This opens up a new tab with the same file, but this time located in your Drive. If you want to continue working after saving, use the file in the new tab. Your notebook will be saved in a folder called Colab Notebooks in your Google Drive by default.

## Load packages
To run the toolbox, we will need a couple of packages including the toolbox itself. So let's install them:

!git clone https://github.com/thilowrona/fault_analysis_toolbox

!pip3 install vtk

!pip3 install git+https://github.com/ulikoehler/cv_algorithms.git

!pip3 install git+https://github.com/MRudey/fault_analysis_toolbox.git

Now we can load the python packages:

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from scipy.spatial import distance_matrix
from tqdm import tqdm

and the functions from the fault analysis toolbox that we want to use:

In [ ]:
from faultanalysistoolbox.edits import array_to_points, label_components
from faultanalysistoolbox.image_processing import guo_hall
from faultanalysistoolbox.plots import plot_components

## Fault extraction

First, we load our data - a strain rate map extracted just below the surface of the model:

In [ ]:
# path = '/content/fault_analysis_toolbox/examples/example-1/NearSurfaceIsotherm_335K_strain_rate.npy'
path = '/home/mrudolf/Nextcloud/GitRepos/fault_analysis_toolbox/examples/1-fault_extraction/NearSurfaceIsotherm_335K_strain_rate.npy'
strain_rate = np.load(
    path
)

Now we can plot it to look the faults in the model

In [ ]:
plt.figure(figsize=(12,12))
plt.title('Strain rate')
plt.imshow(strain_rate, vmin=0)
plt.colorbar()
plt.show()

Next we want to separate the faults from the background using a threshold:

In [ ]:
threshold = np.where(strain_rate > 1.5e-14, 1, 0).astype(np.uint8)

plt.figure(figsize=(12,12))
plt.title('Threshold')
plt.imshow(threshold)
plt.axis('off')
plt.show()

Now we can reduce the areas above the threshold to lines using a skeletonize algorithm:

In [ ]:
skeleton = guo_hall(threshold)

plt.figure(figsize=(12,12))
plt.title('Skeleton')
plt.imshow(skeleton)
plt.axis('off')
plt.show()

Now we can convert these lines to points:

In [ ]:
points = array_to_points(skeleton)

These points become the nodes of our graph G:

In [ ]:
G = nx.Graph()

for node, point in enumerate(points):
    G.add_node(node)
    G.nodes[node]['pos'] = point

Remember a graph is an object consisting only of nodes and edges. Our graph for example looks like this:

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20,12))

axs[0].set_title('Network')
nx.draw(G, 
        pos=nx.get_node_attributes(G, 'pos'), 
        node_size=1,
        ax=axs[0])
axs[0].axis('equal')


axs[1].set_title('Zoom in')
nx.draw(G, 
        pos=nx.get_node_attributes(G, 'pos'), 
        node_size=1,
        ax=axs[1])
axs[1].axis('equal')
axs[1].set_ylim([500, 600])

plt.show()

You can see that the graph only consists of closely spaced points, which are not yet connected. So let's change that!


We calculate the distance between all nodes in a distance matrix and connect the ones close to each other (<1.5 pixels away):

In [ ]:
dm = distance_matrix(points, points) 

# print(str(points.shape[0]) + ' Points')
for n in tqdm(range(points.shape[0]), desc='Connecting Points'):
    # stdout.write("\r%d" % n)
    # stdout.flush()
    for m in range(points.shape[0]):
        if dm[n, m] < 1.5 and n != m:
            G.add_edge(n, m)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20,12))

axs[0].set_title('Graph')
nx.draw(G, 
        pos=nx.get_node_attributes(G, 'pos'), 
        node_size=1,
        ax=axs[0])
axs[0].axis('equal')


axs[1].set_title('Zoom in')
nx.draw(G, 
        pos=nx.get_node_attributes(G, 'pos'), 
        node_size=1,
        ax=axs[1])
axs[1].axis('equal')
axs[1].set_ylim([500, 600])

plt.show()

Now we can see that neighboring nodes are connected by edges (black lines). This allows us to label the nodes connected to one another as components:

In [ ]:
G = label_components(G)

fig, axs = plt.subplots(1, 1, figsize=(12,12))
axs.imshow(strain_rate, 'gray_r', vmin=0)
plot_components(G, axs, label=True)
plt.title('Strain rate with fault network')
plt.show()

When we zoom in, we can see the nodes colored by their component and the edges connecting them:

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(12,12))
axs.imshow(strain_rate, 'gray_r', vmin=0)
plot_components(G, axs, label=False)
axs.set_xlim([250, 450])
axs.set_ylim([400, 600])
plt.title('Strain rate with fault network')
plt.show()

## Structure of the network
Let's have a look at the structure of the fault network (or graph). Remember it only consists of nodes and edges. So let's have a look at the nodes:

In [ ]:
print(G.nodes)

Okay, nothing special here, just a list of the nodes. Let's pick out one:

In [ ]:
print(G.nodes[0])

Alright, we can see the position of the node and the component it belongs to. Let's say we want to give it an extra property, e.g. the strain rate at its location:

In [ ]:
G.nodes[0]['strain_rate'] = strain_rate[int(G.nodes[0]['pos'][1]), int(G.nodes[0]['pos'][0])]

In [ ]:
print(G.nodes[0])

Nice! Let's do that for all nodes:

In [ ]:
for node in G.nodes:
  G.nodes[node]['strain_rate'] = strain_rate[int(G.nodes[node]['pos'][1]), int(G.nodes[node]['pos'][0])]

and plot it:

In [ ]:
fig, ax = plt.subplots(figsize=(20,12))

ax.set_title('Fault network with strain rate')
nx.draw(G, 
        pos=nx.get_node_attributes(G, 'pos'),
        node_color = np.array([G.nodes[node]['strain_rate'] for node in G.nodes]), 
        node_size=1,
        ax=ax)
ax.axis('equal')
plt.show()

Like this we can compute and visualize all kinds of properties on the fault network.

But what about the edges?

In [ ]:
print(G.edges)

Alright, just tuples of nodes. Let's pick one:

In [ ]:
print(G.edges[(0, 5)])

Okay, they have no property yet. Let's calculate its length:

In [ ]:
edge = (0, 5)
G.edges[edge]['length'] = np.linalg.norm(G.nodes[edge[0]]['pos']-G.nodes[edge[1]]['pos'])

In [ ]:
print(G.edges[(0, 5)])

Again, we can do this for all edges:

In [ ]:
for edge in G.edges:
  G.edges[edge]['length'] = np.linalg.norm(G.nodes[edge[0]]['pos']-G.nodes[edge[1]]['pos'])

and plot it:

In [ ]:
fig, ax = plt.subplots(figsize=(20,12))

ax.set_title('Fault network with edge lenghts')
nx.draw(G, 
        pos=nx.get_node_attributes(G, 'pos'),
        edge_color = np.array([G.edges[edge]['length'] for edge in G.edges]), 
        node_size=0.001,
        ax=ax)
ax.axis('equal')
plt.show()

Awesome! That's it. You've extracted your first fault network. In the next tutorial, we will learn how to compute and visualize fault strikes:
https://github.com/thilowrona/fault_analysis_toolbox/blob/master/examples/2-fault_properties/2-fault_properties.ipynb 